In [21]:
import numpy as np
import pandas as pd
import fuzzymatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df = pd.read_csv('title2.csv', encoding = "ISO-8859-1")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63315 entries, 0 to 63314
Data columns (total 12 columns):
id                 63315 non-null int64
title              63315 non-null object
imdb_index         58 non-null object
kind_id            63315 non-null int64
production_year    63315 non-null int64
imdb_id            0 non-null float64
phonetic_code      45054 non-null object
episode_of_id      55876 non-null float64
season_nr          47999 non-null float64
episode_nr         47965 non-null float64
series_years       3492 non-null object
md5sum             63315 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 5.8+ MB


In [3]:
df_right = df[df.kind_id != 7.0]

df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7439 entries, 0 to 63312
Data columns (total 12 columns):
id                 7439 non-null int64
title              7439 non-null object
imdb_index         58 non-null object
kind_id            7439 non-null int64
production_year    7439 non-null int64
imdb_id            0 non-null float64
phonetic_code      7439 non-null object
episode_of_id      0 non-null float64
season_nr          0 non-null float64
episode_nr         0 non-null float64
series_years       3489 non-null object
md5sum             7439 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 755.5+ KB


In [4]:
df_right.describe()

,id,kind_id,production_year,imdb_id,episode_of_id,season_nr,episode_nr
count,7.439000e+03,7439.000000,7439.000000,0.0,0.0,0.0,0.0
mean,2.888508e+06,2.528431,1955.623874,NaN,NaN,NaN,NaN
std,1.292031e+06,0.499225,2.682129,NaN,NaN,NaN,NaN
min,2.514000e+03,2.000000,1950.000000,NaN,NaN,NaN,NaN
25%,1.948836e+06,2.000000,1954.000000,NaN,NaN,NaN,NaN
50%,3.262456e+06,3.000000,1956.000000,NaN,NaN,NaN,NaN
75%,3.940870e+06,3.000000,1958.000000,NaN,NaN,NaN,NaN
max,4.625272e+06,3.000000,1959.000000,NaN,NaN,NaN,NaN


In [5]:
df_left = pd.read_csv('enc.csv', encoding = "ISO-8859-1")
df_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732 entries, 0 to 1731
Data columns (total 12 columns):
Number                 1732 non-null int64
Program Title          1732 non-null object
Program Type           1731 non-null object
Program Genre          1731 non-null object
Network                1704 non-null object
First Air Month        131 non-null float64
First Air Day          123 non-null float64
First Air Year         1732 non-null int64
Last Air Month         2 non-null float64
Last Air Day           2 non-null float64
Last Air Year          895 non-null object
Program Description    1732 non-null object
dtypes: float64(4), int64(2), object(6)
memory usage: 162.5+ KB


In [6]:
df_left["Program Genre"].value_counts()

 Variety            387
 Anthology          274
 Comedy             225
 Game               165
 Adventure          143
 Crime Drama        123
 Drama               88
 Western             85
 Children            71
 Serial              33
 Cartoon             24
 Interview           19
 Women               18
 Human Interest      12
 Documentary         11
 Travel               7
 Mystery              6
 Discussion           6
 Science Fiction      4
 Comedy-Drama         3
 Advice               2
 Contest              2
 ComedyDrama          2
Game                  2
 Music                2
 Testimonial          1
 Adventure            1
 anthology            1
 Cooking              1
 Crime Brama          1
 Variety              1
 Health               1
 Game, Anthology      1
 Musical Drama        1
 Information          1
 Tribute              1
 Children             1
Western               1
 Gomedy               1
 Vatriety             1
 30 min.              1
 Game           

In [7]:
df_right = df_right.astype(str).apply(lambda x: x.str.lower())
df_left = df_left.astype(str).apply(lambda x: x.str.lower())

In [8]:
df_right.drop(df_right.columns[[2,5,7,8,9]], axis=1, inplace=True)

In [9]:
df_right.head()

,id,title,kind_id,production_year,phonetic_code,series_years,md5sum
0,2514,... kurze pause,2,1955,k6212,1955-1956,5d2e160977a1f9ecdb8683b9647fa054
1,4170,10 hobby-minutter,2,1952,h1536,1952-1953,dad9e300b25f72bee9bd8ba5cd47304c
2,5848,100.000 of niets,2,1956,o1532,1956-????,7e4664da716988a7308c8115247feb6d
4,8305,13 demon street,2,1959,d5236,1959-1960,ee82069dbe196c4a272ce7ef7b1877b5
18,9296,13th hour,2,1958,t6,1958-1959,cf74d0ba98aab9b3650ae76cf82d69ac


In [10]:
df_left.drop(df_left.head().columns[[5,6,8,9]], axis=1, inplace=True)

In [11]:
df_left.head()

,Number,Program Title,Program Type,Program Genre,Network,First Air Year,Last Air Year,Program Description
0,5137,mary kay and johnny.,series,comedy,"dumont, nbc, cbs",1947,1950,the marital misadventures of mary kay and joh...
1,7531,the small fry club.,series,children,dumont,1947,1951,"bob emery hosts a program of stories, game co..."
2,1738,crawford mystery theater.,series,game,dumont,1947,1952,the alternate title for public prosecutor (wh...
3,6636,public prosecutor.,series,mystery,"syn., dumont",1947,1952,the syndicated version of the series (1947-48...
4,4323,juvenile jury.,series,children,"nbc, cbs",1947,1955,the program features a panel of five children...


In [12]:
df_right["title"].head()

0       ... kurze pause
1     10 hobby-minutter
2      100.000 of niets
4       13 demon street
18            13th hour
Name: title, dtype: object

In [13]:
df_left["Program Title"].head()

0          mary kay and johnny. 
1           the small fry club. 
2     crawford mystery theater. 
3            public prosecutor. 
4                juvenile jury. 
Name: Program Title, dtype: object

In [19]:
matches = (process.extract(x, df_left, limit=1) for x in df_right)

In [22]:
print(matches)

<generator object <genexpr> at 0x109d66938>


In [130]:
fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on = "Program Title", right_on = "title")

ZeroDivisionError: float division by zero

In [ ]:
# Columns to match on from df_left
left_on = ["Program Title", "First Air Year"]

# Columns to match on from df_right
right_on = ["title", "production_year"]

fuzzymatcher.link_table(df_left, df_right, left_on, right_on, left_id_col = "Number", right_id_col = "id")